# Proyecto: Creación de tablas temporales SQL y consultas SQL en Spark

En la presente práctica veremos cómo:

* Crear Data Frames a partir archivos .csv

* Convertit Data Frames como Tablas temporales tipo SQL

* Hacer consultas con lenguaje SQL en Spark

Algunas funciones que veremos son:

`.registerTempTable('new_table_name_sql')` Transforma un Data Frame como una tabla temporal SQL de nombre 'new_table_name_sql'

`sqlContext.sql('CONSULTA CON CÓDIGO SQL').show()` Realiza una 'CONSULTA CON CÓDIGO SQL' 

In [1]:
# libreria para crear punto de conexión:
from pyspark import SparkContext

# Cargamos libreria para crear Data Frames:
from pyspark.sql import SQLContext

#from pyspark.sql import SparkSession

# Cargamos librerias para crear el schema del DataFrame
from pyspark.sql.types import StructType, StructField

# Cargamos los tipos de datos que usaremos para crear las columnas de los dataframes:
from pyspark.sql.types import  IntegerType, StringType, FloatType

#from pyspark.sql.types import Row

# Importamos functions especiales, entre ellas la función 'col'
from pyspark.sql.functions import *


In [2]:
# Creamos el punto de conexió a Spark (que se ejecutará en mi máquina 'local'):
spark = SparkContext(master='local', appName='manipulando_DataFrames')

In [3]:
# Creamos el contexto para SQL:
sqlContext = SQLContext(spark)

/home/marck/.local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
# Visualizamos el contenido de la carpeta 'Data' 
!ls ./Data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [5]:
# Ruta en donde se encuentran los datos con los que trabajaremos:
path = './Data/'

In [6]:
# Creamos función para eliminar encabezados en RDDs:
def eliminaEncabezado(indice , interador):
    return iter( list(interador)[1:] )

In [7]:
# Número de registros a visualizar:
N=5

## Creación de un DataFrame con los datos del archivo *paises.csv*

In [8]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/paises.csv

id,equipo,sigla
1,30. Februar,AUT
2,A North American Team,MEX
3,Acipactli,MEX
4,Acturus,ARG


In [9]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
paisesSchema = StructType([
    StructField('id',IntegerType(),False),
    StructField('equipo',StringType(),False),
    StructField('sigla',StringType(),False) 
])

# Creamos el DataFrame:
paisesDF = sqlContext.read.schema(paisesSchema).option('header', 'true').csv(path+'paises.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
paisesDF.show(N)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *resultados.csv*

In [10]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/resultados.csv

resultado_id,medalla,deportista_id,juego_id,evento_id
1,NA,1,39,1
2,NA,2,49,2
3,NA,3,7,3
4,Gold,4,2,4


In [11]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
resultadoSchema = StructType([
    StructField('resultado_id',IntegerType(),False),
    StructField('medalla',StringType(),False),
    StructField('deportista_id',IntegerType(),False),
    StructField('juego_id',IntegerType(),False),
    StructField('evento_id',IntegerType(),False)
])

# Creamos el DataFrame:
resultadoDF = sqlContext.read.schema(resultadoSchema).option('header', 'true').csv(path+'resultados.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
resultadoDF.show(N)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



## Creación de un DataFrame con los datos de los archivos *deportista.csv* y *deportista2.csv*

In [12]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deportista.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,0,0,273
4,Edgar Lindenau Aabye,1,34,0,0,278


In [13]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deportista2.csv

67787,Lee BongJu,1,27,167,56,970
67788,Lee BuTi,1,23,164,54,203
67789,Anthony N. Buddy Lee,1,34,172,62,1096
67790,Alfred A. Butch Lee Porter,1,19,186,80,825
67791,Lee ByeongGu,1,22,175,68,970


Notemos que el archivo 'deportista.csv' tiene encabezado y 'deportista2.csv' no tiene

El archivo 'deportista2.csv' es continuación de 'deportista.csv'

Para hacer un sólo Data Frame, haremos lo siguiente: 

1. Tranformamos los archivos.csv a RDDs

2. Eliminamos el encabezado del archivo 'deportista.csv'

3. Unimos los RDDs 'deportista' y 'deportista2

4. Ya que tenemos un sólo RDD lo pasamos a un Data Frame

In [14]:
# Guardamos archivos.csv en RDDs:
# .map(lambda l : l.split(',')) <-- Le asigan formato al contenido de archivos .csv

deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l : l.split(','))

deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l : l.split(','))

In [15]:
# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [16]:
# Eliminamos encabezado al primer RDD :
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [17]:
# Vemos los primeros N registros del segundo RDD:
deportistaOlimpicoRDD2.take(N)

[['67787', 'Lee BongJu', '1', '27', '167', '56', '970'],
 ['67788', 'Lee BuTi', '1', '23', '164', '54', '203'],
 ['67789', 'Anthony N. Buddy Lee', '1', '34', '172', '62', '1096'],
 ['67790', 'Alfred A. Butch Lee Porter', '1', '19', '186', '80', '825'],
 ['67791', 'Lee ByeongGu', '1', '22', '175', '68', '970']]

In [18]:
# Unimos los 2 RDDs: deportistaOlimpicoRDD y deportistaOlimpicoRDD2
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union( deportistaOlimpicoRDD2 )

In [19]:
# Creación de un Data Frame a partir de un RDD:

# Corregimos los tipos de dato en cada columna del RDD: 
deportistaOlimpicoRDD = \
    deportistaOlimpicoRDD.map(lambda x: ( int(x[0]), x[1],  int(x[2]), int(x[3]), int(x[4]), float(x[5]), int(x[6])  ) )


# A continuación creamos el schema, es decir, la estructura (columnas) del Data Frame donde pondremos los datos del RDD:
schema = StructType([
    StructField('deportista_id', IntegerType(), False ),
    StructField('nombre', StringType(), False ),
    StructField('genero', IntegerType(), False ),
    StructField('edad', IntegerType(), False ),
    StructField('altura', IntegerType(), False ),
    StructField('peso', FloatType(), False ),
    StructField('equipo_id', IntegerType(), False )
    ])

# StructField <-- instrucción para crear una columna, se debe especificar el tipo de dato que almacenará
# En caso de que el campo pueda ser nulo ponemos 'True' de lo contrario ponemos 'False'


# Creamos el DataFrame a partir del RDD:
deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)


# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
deportistaOlimpicoDF.show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [20]:
# Vemos los primeros N registros ordenadas de forma ascendente 
# con respecto a los valores de la columna 'edadAlJugar'

deportistaOlimpicoDF.sort('edad').show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|          133|           Franz Abb|     1|   0|     0| 0.0|      399|
|          167|Ould Lamine Abdallah|     1|   0|     0| 0.0|      362|
|           66|     Mohamed Abakkar|     1|   0|   156|48.0|     1003|
|          163|     Ismail Abdallah|     1|   0|     0| 0.0|     1095|
|          139|George Ioannis Abbot|     1|   0|     0| 0.0|     1043|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [21]:
# Nos quedamos con registros donde los valores de la columna 'edad' sean distintos de cero
deportistaOlimpicoDF = deportistaOlimpicoDF.filter( (deportistaOlimpicoDF['edad'] !=0) )

In [22]:
# Mostramos los primeros registros ordenados con respecto a los valores de la columna 'edad'
deportistaOlimpicoDF.sort('edad').show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|        71691|  Dimitrios Loundras|     1|  10|     0| 0.0|      333|
|        52070|        Etsuko Inada|     2|  11|     0| 0.0|      514|
|        40129|    Luigina Giavotti|     2|  11|     0| 0.0|      507|
|        37333|Carlos Bienvenido...|     1|  11|     0| 0.0|      982|
|        47618|Sonja Henie Toppi...|     2|  11|   155|45.0|      742|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



## Registramos Data Frames como tabalas temporales tipo SQL

In [23]:
# Registro de Data Frames:

resultadoDF.registerTempTable('resultado')

deportistaOlimpicoDF.registerTempTable('deportista')

paisesDF.registerTempTable('paises')

/home/marck/.local/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


## A continuación usamos código tipo SQL

In [24]:
sqlContext.sql('SELECT * FROM deportista').show()

+-------------+--------------------+------+----+------+-----+---------+
|deportista_id|              nombre|genero|edad|altura| peso|equipo_id|
+-------------+--------------------+------+----+------+-----+---------+
|            1|           A Dijiang|     1|  24|   180| 80.0|      199|
|            2|            A Lamusi|     1|  23|   170| 60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0|  0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0|  0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185| 82.0|      705|
|            6|     Per Knut Aaland|     1|  31|   188| 75.0|     1096|
|            7|        John Aalberg|     1|  31|   183| 72.0|     1096|
|            8|Cornelia Cor Aalt...|     2|  18|   168|  0.0|      705|
|            9|    Antti Sami Aalto|     1|  26|   186| 96.0|      350|
|           10|Einar Ferdinand E...|     1|  26|     0|  0.0|      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182| 76.5|   

In [25]:
sqlContext.sql(''' 
SELECT medalla, equipo, sigla FROM resultado AS r
JOIN deportista AS d
ON r.deportista_id = d.deportista_id
JOIN paises AS p
ON p.id = d.equipo_id
WHERE medalla != 'NA'
ORDER BY sigla DESC
'''
).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [26]:
spark

<SparkContext master=local appName=manipulando_DataFrames>

In [27]:
# Cerramos sesión para liberar memoria:
spark.stop()